<a href="https://colab.research.google.com/github/villasen/colab_notebooks/blob/master/STM32F746_model_ARM_script_Urban.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/villasen/small-urban-sound-dataset

Cloning into 'small-urban-sound-dataset'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 35377 (delta 78), reused 89 (delta 43), pack-reused 35251
Receiving objects: 100% (35377/35377), 977.47 MiB | 29.01 MiB/s, done.
Resolving deltas: 100% (12710/12710), done.
Checking out files: 100% (43960/43960), done.


In [0]:
# Clone the repository from GitHub
!git clone -q https://github.com/tensorflow/tensorflow
# Check out a commit that has been tested to work
# with the build of TensorFlow we're using
!git -c advice.detachedHead=false -C tensorflow checkout 17ce384df70

In [0]:
!pip uninstall -y tensorflow tensorflow_estimator
!pip install -q tf-estimator-nightly==1.14.0.dev2019072901 tf-nightly-gpu==1.15.0.dev20190729

Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
Uninstalling tensorflow-estimator-1.15.1:
  Successfully uninstalled tensorflow-estimator-1.15.1
     |████████████████████████████████| 501kB 3.4MB/s 
     |████████████████████████████████| 406.6MB 42kB/s 
     |████████████████████████████████| 3.8MB 53.4MB/s 


In [28]:
import os
os.environ["WANTED_WORDS"] = "footsteps,glass_breaking,door_knock,clapping,dog,clock_alarm"

#os.environ["WANTED_WORDS"] = "yes,no"  
os.environ["TRAINING_STEPS"]="15000,3000"
#os.environ["TRAINING_STEPS"]="8000,2000"
os.environ["LEARNING_RATE"]="0.001,0.0001"

total_steps = sum(map(lambda string: int(string),
                  os.environ["TRAINING_STEPS"].split(",")))
os.environ["TOTAL_STEPS"] = str(total_steps)

# Print the configuration to confirm it
!echo "Training these words: ${WANTED_WORDS}"
!echo "Training steps in each stage: ${TRAINING_STEPS}"
!echo "Learning rate in each stage: ${LEARNING_RATE}"
!echo "Total number of training steps: ${TOTAL_STEPS}"

Training these words: footsteps,glass_breaking,door_knock,clapping,dog,clock_alarm
Training steps in each stage: 15000,3000
Learning rate in each stage: 0.001,0.0001
Total number of training steps: 18000


In [0]:
!rm -rf /content/retrain_logs
# Load TensorBoard
%reload_ext tensorboard
%tensorboard --logdir /content/retrain_logs

In [0]:
!rm -rf /content/small-urban-sound-dataset/

In [0]:
!python tensorflow/tensorflow/examples/speech_commands/train.py \
--data_url= none --feature_bin_count=10 \
--model_architecture=tiny_embedding_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --silence_percentage=20 --unknown_percentage=25 \
--quantize=1 --verbosity=INFO --how_many_training_steps=${TRAINING_STEPS} \
--learning_rate=${LEARNING_RATE} --summaries_dir=/content/retrain_logs \
--train_dir=/content/speech_commands_train \
--data_dir=/content/small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz \

python3: can't open file 'tensorflow/tensorflow/examples/speech_commands/train.py': [Errno 2] No such file or directory


In [25]:
%env MODEL_SIZE=1 64 10 4 2 2 

env: MODEL_SIZE=1 64 10 4 2 2


In [29]:
!python /content/small-urban-sound-dataset/speech_commands/train.py \
--data_url= none --eval_step_interval=100  --feature_bin_count=10 \
--model_architecture=martin_tiny_urban_model --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --silence_percentage=10 --unknown_percentage=10 \
--quantize=1 --verbosity=INFO --how_many_training_steps=${TRAINING_STEPS} \
--learning_rate=${LEARNING_RATE} --summaries_dir=/content/retrain_logs \
--train_dir=/content/speech_commands_train  \
--data_dir=/content/small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz \
--model_size_info $MODEL_SIZE

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

2020-02-09 06:57:14.673995: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-02-09 06:57:14.693821: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-02-09 06:57:14.862706: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-02-09 06:57:14.863490: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1a40a00 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=mfcc --feature_bin_count=10 \
--wanted_words=${WANTED_WORDS} --quantize=1 --output_file=/content/2labels_40mfcc.pb \
--start_checkpoint=/content/speech_commands_train/tiny_conv.ckpt-${TOTAL_STEPS}

In [0]:
!python /content/small-urban-sound-dataset/speech_commands/freeze.py --feature_bin_count=10 \
--model_architecture=martin_tiny_urban_model --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --quantize=1 --output_file=/content/6labels_10mfcc.pb \
--start_checkpoint=/content/speech_commands_train/martin_tiny_urban_model.ckpt-${TOTAL_STEPS}

In [0]:
!python /content/small-urban-sound-dataset/speech_commands/label_wav.py \
--graph=/content/6labels_40mfcc.pb \
--labels=/content/speech_commands_train/tiny_conv_labels.txt \
--how_many_labels=6 \
--wav=/content/small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz/glass_breaking/glass_breaking-B-4-204777-B.wav

In [0]:
!python /content/small-urban-sound-dataset/speech_commands/label_wav.py \
--graph=/content/6labels_10mfcc.pb \
--labels=/content/speech_commands_train/martin_tiny_urban_model_labels.txt \
--how_many_labels=6 \
--wav=/content/small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz/glass_breaking/glass_breaking-B-4-204777-B.wav

In [0]:
import os.path
from os import path



In [0]:
import os.path
from os import path

input_files="/content/input_files/"

os.mkdir(input_files)


In [0]:
DATA_PATH= '/content/small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz/'

labels = os.listdir('small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz/')
print(labels)

In [0]:
!rm -r clapping-D-1-105224-A.wav
!rm -r /content/dog-A-4-192236-A.wav
!rm -r /content/door_knock-A-3-182025-A.wav
!rm -r /content/footsteps-B-2-209471-A.wav
!rm -r /content/glass_breaking-A-2-106881-A.wav

In [0]:
path = DATA_PATH
i=0
for label in labels:
  
  output_file=[]
  #utput_file_tmp=[]
  if label=='footsteps' or label=='glass_breaking' or label=='door_knock' or label=='clock_alarm' \
  or label=='clapping' or label=='dog': 
    files = os.listdir(path + '/' + label)
    os.mkdir("input_files/" + label)
    my_files_tmp = files[i]
    my_files = my_files_tmp[:-4]
    #print(my_files)
    #os.mkdir(my_files[i])

    input_files = [path + label + '/' + testfile for testfile in os.listdir(path + '/' + label)]
    
    #print(input_files[i])
    #print(os.listdir(path + '/' + label))
    output_file_tmp = os.listdir(path + '/' + label)
    
    output_file = my_files
    output_cfile = "/content/input_files/" + label + "/" + output_file + ".cc"
    
    os.environ["INPUT_FILES"]=input_files[i]
    os.environ["OUTPUT_FILES"]=output_cfile


    # !python /content/small-urban-sound-dataset/speech_commands/wav_to_features.py  \
    # --sample_rate=16000 \
    # --window_size_ms=30 \
    # --feature_bin_count=40 \
    # --window_stride_ms=20 \
    # --quantize=True  \
    # --preprocess="micro" \
    # --input_wav=${INPUT_FILES} \
    # --output_c_file=${OUTPUT_FILES} 
    i = i + 1

In [0]:
!python /content/small-urban-sound-dataset/speech_commands/wav_to_features.py  \
--sample_rate=16000 \
--window_size_ms=30 \
--feature_bin_count=10 \
--window_stride_ms=20 \
--quantize=True  \
--preprocess="micro" \
--input_wav="/content/small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz/dog/dog-A-2-114587-A.wav" \
--output_c_file="/content/input_files/dog-micro.cc"


In [0]:
import numpy as np
!python /content/small-urban-sound-dataset/speech_commands/wav_to_features_for_RPI.py  \
--sample_rate=16000 \
--window_size_ms=30 \
--feature_bin_count=10 \
--window_stride_ms=20 \
--quantize=True  \
--preprocess="micro" \
--input_wav="/content/small-urban-sound-dataset/ESC-50-split/ESC-50_split_data_set_one-second-16KHz/dog/dog-A-2-114587-A.wav" \
--output_c_file="/content/input_files/dog-A-2-114587-A.npy"




In [0]:
!toco \
--graph_def_file=/content/6labels_10mfcc.pb --output_file=/content/6labels_10mfcc.tflite \
--input_shapes=1,49,10,1 --input_arrays=Reshape_2 --output_arrays='labels_softmax' \
--inference_type=QUANTIZED_UINT8 --mean_values=0 --std_dev_values=9.8077

In [0]:
import os
model_size = os.path.getsize("/content/6labels_10mfcc.tflite")
print("Model is %d bytes" % model_size)

In [0]:
#Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i /content/6labels_10mfcc.tflite > /content/6labels_10mfcc.cc
# Print the source file
!cat /content/6labels_10mfcc.cc

In [0]:

!rm tiny_conv.cc
!rm tiny_conv.pb
!rm tiny_conv.tflite
!rm -rf sample_data/

In [0]:
!rm -rf speech_commands_train/
!rm -rf retrain_logs/